## Refer: [IFD](https://github.com/tianyi-lab/Superfiltering)
$$\mathrm{IFD}_\theta(Q,A)=\frac{\mathrm{PPL}_\theta(A|Q)}{\mathrm{PPL}_\theta(A)}$$


In [ ]:
import os
import numpy as np 
import pandas as pd 
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, save_npz, load_npz, hstack
import lightgbm as lgb
from tqdm import tqdm
import gensim
import itertools
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from transformers import DebertaV2Tokenizer, DebertaV2Model
from transformers import set_seed, AutoModelForCausalLM, AutoTokenizer
import torch
import joblib
import unicodedata
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [ ]:
MAX_LENGTH = 1024
deberta_path = "/kaggle/input/debertav3base"
gpt_path = "/kaggle/input/qwen2-1-5b-instruct"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42
set_seed(SEED)

In [ ]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")
vectorize_on_train_and_test = True
#quick_test for training on small part of train data (and not using bunch of GPU on submit)
#(if this is on - saved models won't be fully trained)
quick_test = True
quick_test_items = 1000
#automatically disable quick_test if we detect actual test data... (assures full training when scoring)
if (len(test)) > 3:quick_test = False
if quick_test: train = train.head(quick_test_items)

def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)
print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))

target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]
train['label'] = train[target_columns].idxmax(axis=1) 
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]
train.head(3)

### TF-IDF

In [ ]:
train_text = train[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)
test_text = test[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)

if vectorize_on_train_and_test:
    vector_fit_text = pd.concat([train_text, test_text], axis=0).reset_index(drop=True)
else:
    vector_fit_text = train_text

In [ ]:
def custom_tokenizer(text):
    return re.findall(r'[^\W]+', text)

#word-level vectorizer
tfidf_word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 5),
    tokenizer=custom_tokenizer,
    token_pattern=None,
    strip_accents='unicode',
    min_df=4,
    max_features=300
)

#char-level vectorizer
tfidf_char_vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1, 5), 
    max_features=1000, 
    min_df=4
)

def batch_process(texts, batch_size):
    for i in range(0, len(texts), batch_size):
        yield texts[i:i + batch_size]

#doing in batches so we can see progress
batch_size = 1000
for batch in tqdm(batch_process(vector_fit_text, batch_size), total=np.ceil(len(vector_fit_text) / batch_size)):
    if len(batch) >= tfidf_word_vectorizer.min_df:
        tfidf_word_vectorizer.fit(batch)
    if len(batch) >= tfidf_char_vectorizer.min_df:
        tfidf_char_vectorizer.fit(batch)
        
def get_tfidf_vectors(df):
    vectorized_columns = []
    for column in columns_to_vectorize:
        vectorized_columns.append(tfidf_word_vectorizer.transform(df[column]))
        vectorized_columns.append(tfidf_char_vectorizer.transform(df[column]))
    return hstack(vectorized_columns)

tfidf_train_vectors = get_tfidf_vectors(train)

### LEN

In [ ]:
def has_none(vals) -> int:
    # some responses contains null and probably they are useful for prediction
    return int(any(val is None for val in vals))

def str_length(vals) -> int:
    length = 0
    for val in vals:
        if isinstance(val, str):
            length += len(val)
    return length

def get_length_features(data: pd.DataFrame):
    length_feature_array = []
    length_feature_array.append(data["response_a"].apply(str_length))
    length_feature_array.append(data["response_b"].apply(str_length))
    length_feature_array.append(length_feature_array[0] - length_feature_array[1])
    length_feature_array.append((length_feature_array[0] + length_feature_array[1]) / 2)
    length_feature_array.append((length_feature_array[0] / length_feature_array[1]))
    length_feature_array.append(data["response_a"].apply(has_none))
    length_feature_array.append(data["response_b"].apply(has_none))
    length_feature_array.append(data["response_a"].apply(has_none) - data["response_b"].apply(has_none))
    length_feature_array = np.array(length_feature_array).reshape(len(length_feature_array), -1)
    length_feature_array = np.transpose(length_feature_array, (1, 0))
    return length_feature_array
train_length_features = get_length_features(train)
print(train_length_features.shape)

### IFD

In [ ]:
def get_ifd_features(data: pd.DataFrame):
    model = AutoModelForCausalLM.from_pretrained(gpt_path, torch_dtype=torch.bfloat16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(gpt_path)
    model.eval()
    def get_ppl_features(output, instruct=''):
        try:
            answer_start_index = 0
            if instruct != '':
                answer_start_index = tokenizer.encode(instruct, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).shape[1]
            input_ids = tokenizer.encode(instruct + output, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(device)
            labels = input_ids.clone()
            labels[:, :answer_start_index] = -100
            with torch.no_grad():
                outputs = model(input_ids, labels=labels)
            perplexity = torch.exp(outputs.loss)
            return perplexity.to('cpu').item()
        except:
            return 0
        
    ppl_ifd_feature_array = []
    for i in tqdm(range(len(data)), desc="Scoring ifd"):
        instruct = data['prompt'][i]
        output_a = data['response_a'][i]
        output_b = data['response_b'][i]
        
        ppl_ca_a, ppl_da_a = get_ppl_features(output_a, instruct), get_ppl_features(output_a)
        ppl_ca_b, ppl_da_b = get_ppl_features(output_b, instruct), get_ppl_features(output_b)
        try:
            ifd_a = ppl_ca_a / ppl_da_a
        except ZeroDivisionError:
            ifd_a = 0
        try:
            ifd_b = ppl_ca_b / ppl_da_b
        except ZeroDivisionError:
            ifd_b = 0
        ppl_ifd_feature_array.append([ppl_ca_a, ppl_da_a, ifd_a, ppl_ca_b, ppl_da_b, ifd_b, ifd_a - ifd_b, ifd_a - ifd_b > 0])
    ppl_ifd_feature_array = np.array(ppl_ifd_feature_array).reshape(len(ppl_ifd_feature_array), -1)
    return ppl_ifd_feature_array
ppl_ifd_features = get_ifd_features(train)
print(ppl_ifd_features.shape)

### Combine

In [ ]:
tfidf_train_vectors = csr_matrix(tfidf_train_vectors)*0.2
ppl_ifd_features_csr = csr_matrix(ppl_ifd_features)*0.7
train_length_features_csr = csr_matrix(train_length_features)*0.1
combined_train_vectors = hstack([tfidf_train_vectors, train_length_features_csr, ppl_ifd_features_csr])
print(combined_train_vectors.shape)
print("Vectorizing test text...")
tfidf_test_vectors = get_tfidf_vectors(test)
tfidf_test_vectors_csr = csr_matrix(tfidf_test_vectors)*0.2
test_ppl_ifd_features = get_ifd_features(test)
test_ppl_ifd_features_csr = csr_matrix(test_ppl_ifd_features)*0.7
test_length_features = get_length_features(test)
test_length_features_csr = csr_matrix(test_length_features)*0.1
combined_test_vectors = hstack([tfidf_test_vectors_csr, test_length_features_csr, test_ppl_ifd_features_csr]) 
print("Done!")

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import joblib

max_estimators = 1000
early_stopping_limit = 100

# Data preparation
X = combined_train_vectors
y_encoded = train['label'].values

# LightGBM parameters
params = {
    'n_estimators': max_estimators,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'random_state': 42,
    'learning_rate': 0.03,
    'verbose': -1  # keep logs quiet
}

# Create the model
model = lgb.LGBMClassifier(**params)

# 5-fold cross-validation
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logloss_scores = []
accuracy_scores = []
test_pred_list = []

for fold, (train_indices, val_indices) in enumerate(stratified_k_fold.split(X, y_encoded)):
    print(f"\nFold {fold + 1}")
    X_train_fold, X_val_fold = X[train_indices], X[val_indices]
    y_train_fold, y_val_fold = y_encoded[train_indices], y_encoded[val_indices]

    def callback(env):
        if env.iteration % 10 == 0: print ("Iteration:", env.iteration, "\tLog Loss:", env.evaluation_result_list[0][2])

    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_limit), callback]
    )

    y_pred_proba_fold = model.predict_proba(X_val_fold)
    logloss_fold = log_loss(y_val_fold, y_pred_proba_fold)
    logloss_scores.append(logloss_fold)
    print(f"Log Loss: {logloss_fold}")
    
    y_pred_fold = np.argmax(y_pred_proba_fold, axis=1)
    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)
    accuracy_scores.append(accuracy_fold)
    print(f"Accuracy: {accuracy_fold}")

    test_pred_list.append(model.predict_proba(combined_test_vectors[-test.shape[0]:]))

# Calculate and print average scores
average_logloss = np.mean(logloss_scores)
average_accuracy = np.mean(accuracy_scores)
print(f"\nAverage Log Loss: {average_logloss}")
print(f"Average Accuracy: {average_accuracy}")

In [ ]:
preds_test = np.mean(test_pred_list, axis=0)
submission = pd.DataFrame({
    'id': test["id"],
    'winner_model_a': preds_test[:, 0],
    'winner_model_b': preds_test[:, 1], 
    'winner_tie': preds_test[:, 2]
})
submission.to_csv('submission.csv', index=False)
display(submission)